# NDVI for Sentinel

The following code is able to extract NDVI data from Sentinel scenes using the one of the area with data available between two dates

First we load the needed Python libraries

In [ ]:
import pystac
import pystac_client
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import numpy as np
import hvplot.pandas
from configparser import ConfigParser
from datetime import datetime
from datetime import date
import shapely.geometry
from ipyleaflet import Map, TileLayer, GeoJSON, FullScreenControl
import stackstac
from geogif import gif, dgif
import rasterio
import rasterio.mask
from rasterio.merge import merge
from rasterio.rio.sample import sample
import pyproj
from pyproj import Transformer
from pyproj.database import query_utm_crs_info
from lib.functions import xarray_to_rasterio, mask_raster_with_geometry, create_memory_dataset, utm_from_extent, nearest

In [ ]:
print(pystac.version.get_stac_version())
print(pystac_client.__version__)

# Get database data

Setting some variables used in the following code

In [ ]:
SDATE='2020-01-01'
EDATE='2020-01-31'

Reading configuration file with info about the connection

In [ ]:
config = ConfigParser()
config.read("setting.ini")
dbsett = config["eurodeer_db"]
eu_bbox = ["-24.2","35.2","43.4","71.0"]
# create connection with eurodeer_db
db_connection_url = "postgresql://{us}:{pas}@{host}:{port}/{db}".format(us=dbsett['user'],
                                                                      pas=dbsett['password'],
                                                                      host=dbsett['host'],
                                                                      port=dbsett['port'],
                                                                      db=dbsett["db"]
                                                                     )

We now create a GeoPandas object with all the study areas and their bounding box

In [ ]:
con = create_engine(db_connection_url)
eu_sql = "select study_areas_id, study_name, CASE WHEN geom is not null THEN geom WHEN geom_vhf is not null THEN geom_vhf END as geom from main.study_areas"
eu_df = gpd.GeoDataFrame.from_postgis(eu_sql, con)
eu_df.head()

But it is more useful to get only study areas within the start and end dates, set before.

In [ ]:
indate_sql = "select distinct(sa.*) from main.animals as ani, main.study_areas as sa, (select distinct(animals_id) from main.gps_data_animals as gps where acquisition_time >= '{SDATE}' and acquisition_time < '{EDATE}') as q where q.animals_id = ani.animals_id and ani.study_areas_id=sa.study_areas_id".format(SDATE=SDATE, EDATE=EDATE)
indate_df = gpd.GeoDataFrame.from_postgis(indate_sql, con)
indate_df.head()

In [ ]:
eu_df = indate_df

For the test we are going to use Cembra area 

In [ ]:
studyarea = eu_df[eu_df["study_areas_id"] == 34]

In [ ]:
studyarea.hvplot(tiles='OSM')

And we get the bounding box of the geometry

In [ ]:
bbox = studyarea.geom.envelope.bounds
print(bbox.values[0])

We get the EPSG code to use from the bounding box

In [ ]:
EPSG = utm_from_extent(bbox.values[0])
print(EPSG)

Now we print the study area in hectares

In [ ]:
studyarea_m2 = float(studyarea.to_crs(f"EPSG:{EPSG.to_epsg()}").geom.area)
studyarea_ha = studyarea_m2 / 1000000
print(studyarea_ha)

reproject the bbox to the corresponding UTM values; **pay attention that the newer Proj version for EPSG:4326 that y,x as input and return x,y as output**

In [ ]:
## WRONG since now 4326 is lat, long
transformer = Transformer.from_crs("epsg:4326", f'epsg:{EPSG.to_epsg()}')
x1, y1 = transformer.transform(bbox.minx.values[0], bbox.miny.values[0])
x2, y2 = transformer.transform(bbox.maxx.values[0], bbox.maxy.values[0])
bbox_utm = (x1, y1, x2, y2)
print(bbox_utm)

In [ ]:
x1, y1 = transformer.transform(bbox.miny.values[0], bbox.minx.values[0])
x2, y2 = transformer.transform(bbox.maxy.values[0], bbox.maxx.values[0])
bbox_utm_reverse = (x1, y1, x2, y2)
print(bbox_utm_reverse)

Get the GPS point data for the selected times

In [ ]:
gps_sql = "select gps.* from main.gps_data_animals as gps where gps.animals_id in (select animals_id from main.animals where study_areas_id={na} and acquisition_time >= '{ts}' and acquisition_time < '{te}');".format(na=studyarea.study_areas_id.values[0], ts=SDATE, te=EDATE)
print(gps_sql)
studyarea_points = gpd.GeoDataFrame.from_postgis(gps_sql, con, geom_col='geom')
len(studyarea_points)

In [ ]:
#studyarea_points.geom.is_empty.count()

In [ ]:
#studyarea_points[~(studyarea_points.is_empty | studyarea_points.isna())]

In [ ]:
print(min(studyarea_points['acquisition_time']))
print(max(studyarea_points['acquisition_time']))

Create date range for the pystac Client

In [ ]:
mindate = min(studyarea_points['acquisition_time']).date().strftime('%Y-%m-%d')
maxdate = max(studyarea_points['acquisition_time']).date().strftime('%Y-%m-%d')
date_range = "{mi}/{ma}".format(mi=mindate, ma=maxdate)
print(date_range)

Get all the dates where there are some data (useful to avoid days without data)

In [ ]:
studyarea_points_dates = sorted(set(studyarea_points.acquisition_time.dt.strftime("%Y-%m-%d").to_list()))
studyarea_points_dates

## Working with STAC repository 

The **SpatioTemporal Asset Catalog** (STAC) specification provides a common language to describe a range of geospatial information, so it can more easily be indexed and discovered. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time. 

The goal is for all providers of spatiotemporal assets (Imagery, SAR, Point Clouds, Data Cubes, Full Motion Video, etc) to expose their data as SpatioTemporal Asset Catalogs (STAC), so that new code doesn't need to be written whenever a new data set or API is released.

https://stacspec.org/

### Working with AWS storage

Inizialize the python STAC Client

In [ ]:
aws_catalog = pystac_client.Client.open("https://earth-search.aws.element84.com/v0")

Print all the available catalogs

In [ ]:
for i in aws_catalog.get_all_collections():print(i)

Search for tiles in Sentinel 2 L2A cogs dataset where the bounding box is contained in our study area and in the dates range used

**Cloud Optimized GeoTIFF** (COGS) is a regular GeoTIFF file, aimed at being hosted on a HTTP file server, with an internal organization that enables more efficient workflows on the cloud. It does this by leveraging the ability of clients issuing ​HTTP GET range requests to ask for just the parts of a file they need.

In [ ]:
aws_sent = aws_catalog.search(collections=["sentinel-s2-l2a-cogs"], bbox=bbox.values[0],
    datetime=date_range)

Get all the items founded and print the number

In [ ]:
aws_sent_items = aws_sent.get_all_items()
print(len(aws_sent_items))

Let's see what is an item

In [ ]:
item=aws_sent_items.items[0]

In [ ]:
item.assets

We are going to create a STAC object with all the STAC items with only the band RED and NIR clipping the data in the bounding box

In [ ]:
ds = stackstac.stack(aws_sent_items, assets=["B04", "B08"], resolution=10, epsg=EPSG, bounds_latlon=bbox.values.tolist()[0])

In [ ]:
ds

Calculate NDVI dataset and bring the attributes from original dataset 

In [ ]:
nir, red = ds.sel(band="B08"), ds.sel(band="B04")
ndvi = (nir - red) / (nir + red)

In [ ]:
ndvi.attrs = ds.attrs
ndvi

Now we get the uniq dates for the NDVI dataset

In [ ]:
ndvi_days = set(sorted(list(ndvi.time.dt.date.values)))

In [ ]:
ndvi_days

For each NDVI day we assign all the gps dates to be queried 

In [ ]:
assigned_dates = {}
#we create first the dictionary
for i in ndvi_days:
    assigned_dates[i.strftime("%Y-%m-%d")] = [] 

In [ ]:
# and later we fill it using the nearest function
for i in studyarea_points_dates:
    idate = datetime.strptime(i, "%Y-%m-%d").date()
    near = nearest(ndvi_days, idate)
    assigned_dates[near.strftime("%Y-%m-%d")].append(idate)

In [ ]:
assigned_dates

For each NDVI map date we process the data, creating the mosaic, if needed, and query the map with the points for the selected time range

In [ ]:
test ={'2020-01-27': [date(2020, 1, 26),
  date(2020, 1, 27),
  date(2020, 1, 28)],
 '2020-01-29': [date(2020, 1, 29), date(2020, 1, 30)],}

In [ ]:
for k,v in test.items():
    #get NDVI maps for the selected date
    maps_merge = ndvi[ndvi.time.dt.date == datetime.strptime(k, "%Y-%m-%d").date()]
    daily_raster = []
    #with more maps will merge them 
    if len(maps_merge) > 1:
        for inmerge in maps_merge:
            daily_raster.append(create_memory_dataset(inmerge, EPSG, ndvi.attrs['transform']))
        merged, mergedtransf = merge(daily_raster)
        inmem = create_memory_dataset(merged, EPSG, mergedtransf)
    else:
        inmem = create_memory_dataset(maps_merge[0], EPSG, ndvi.attrs['transform'])
    # get data points beetween minimum and maximum date connected to the date of NDVI map
    print(type(v))
    daypoints=studyarea_points[(studyarea_points.acquisition_time.dt.date >= min(v)) & (studyarea_points.acquisition_time.dt.date < max(v))]
    daypointsxy = {}
    # we need to transform the points from LatLong to UTM
    for i,x,y in zip(daypoints.gps_data_animals_id, daypoints.geom.x , daypoints.geom.y):
        daypointsxy[i] = transformer.transform(y,x)
    x=0
    keys = list(daypointsxy.keys())
    #get points data from the raster
    for val in inmem.sample(daypointsxy.values()):
        if len(val) == 1:
            print(keys[x], val[0])
        else:
            print("No data for ID {}".format(key[x]))
        x += 1

It could be also possible to save the map to raster and use them in different program

In [ ]:
for k in assigned_dates.keys():
    maps_merge = ndvi[ndvi.time.dt.date == datetime.strptime(k, "%Y-%m-%d").date()]
    daily_raster = []
    #with more maps will merge them 
    if len(maps_merge) > 1:
        for inmerge in maps_merge:
            daily_raster.append(create_memory_dataset(inmerge, EPSG, ndvi.attrs['transform']))
        out_image, out_transform = merge(daily_raster)
    else:
        out_image, out_transform = mask_raster_with_geometry(maps_merge[0],
                                                             studyarea.to_crs(EPSG).geom,
                                                             crop=True, nodata=np.nan)
    out_meta = ds.attrs
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "crs": EPSG
                    })
    bandtime = maps_merge[0].time.dt.date.values
    print(out_image.dtype)
    
    with rasterio.open(f"/tmp/ndvi{bandtime}.tif", "w", dtype=out_image.dtype, count=1, nodata=None, **out_meta) as dest:
        dest.write(out_image)
        print(f"/tmp/ndvi{bandtime}.tif saved")

In [ ]:
aws_sent = aws_catalog.search(collections=["sentinel-s2-l2a-cogs"], bbox=bbox.values[0],
    datetime=date_range)

## Known problem

* **missing data**: clouds and not constant fligh area

* **time**: it will take long time to process all the data, one of the most time consuming activity is the point query with raster

## Improvement

* test other solution to query the points (passing by GRASS GIS for example)

* extract a script from the above lines to be run in multiple process 

## Other resources

* **Landsat** data to increse the number of maps and to reduce the probability to find clouds